In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import jaccard_score

# Load the grayscale fingerprint image
image = cv2.imread('img1.png', cv2.IMREAD_GRAYSCALE)
if image is None:
    raise FileNotFoundError("Error: 'img1.png' not found.")

# Convert to binary
_, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Define kernel for morphological operations
kernel = np.ones((3, 3), np.uint8)

# Apply morphological operations
operations = {
    "Eroded": cv2.erode(binary, kernel, iterations=1),
    "Dilated": cv2.dilate(binary, kernel, iterations=1),
    "Opened": cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel),
    "Closed": cv2.dilate(cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel), kernel, iterations=1),

}

# Evaluation function
def evaluate(img, gt):
    result_flat = img.flatten() // 255
    gt_flat = gt.flatten() // 255
    ssim_score = ssim(gt, img)
    iou_score = jaccard_score(gt_flat, result_flat)
    return ssim_score, iou_score

# Evaluate all processed results
scores = {}
for name, img in operations.items():
    ssim_score, iou_score = evaluate(img, binary)
    scores[name] = (ssim_score, iou_score)
    print(f"{name} -> SSIM: {ssim_score:.4f}, IoU: {iou_score:.4f}")

# Identify the best match based on SSIM and IoU
best_name, (best_ssim, best_iou) = max(scores.items(), key=lambda x: (x[1][0], x[1][1]))
best_img = operations[best_name]
print(f"\n✅ Most accurate result for fingerprint recognition: {best_name} (SSIM: {best_ssim:.4f}, IoU: {best_iou:.4f})")

# Annotate the best result
annotated = cv2.cvtColor(best_img, cv2.COLOR_GRAY2BGR)
cv2.putText(annotated, f"Best: {best_name}", (10, 25),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

# Convert original to BGR for side-by-side comparison
original_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
cv2.putText(original_color, "Original", (10, 25),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

# Stack images side by side
comparison = np.hstack((original_color, annotated))

# Show comparison
cv2.imshow("Fingerprint Comparison: Original vs Best Processed", comparison)

# Optionally show individual processed images
for name, img in operations.items():
    cv2.imshow(name, img)

cv2.waitKey(0)
cv2.destroyAllWindows()


Eroded -> SSIM: 0.3159, IoU: 0.5024
Dilated -> SSIM: 0.3904, IoU: 0.5912
Opened -> SSIM: 0.5746, IoU: 0.7687
Closed -> SSIM: 0.3904, IoU: 0.5912

✅ Most accurate result for fingerprint recognition: Opened (SSIM: 0.5746, IoU: 0.7687)
